In [2]:
import numpy as np
import pandas as pd
import riskfolio as rp
import matplotlib.pyplot as plt
import yfinance as yf

In [77]:
tickers = ['CSSPX.MI', 'IEUR', 'WSML.L', 'VFEA.MI', 'EWJ', 'VGEK.DE']
start_date = '2000-01-01'

prices = yf.download(tickers, start=start_date, auto_adjust=True)['Close']

[*********************100%***********************]  6 of 6 completed
[*********************100%***********************]  6 of 6 completed


In [78]:
returns = prices.pct_change().dropna()
monthly_returns = returns.resample('ME').agg(lambda x: (1 + x).prod() - 1)

C:\Users\kutil\AppData\Local\Temp\ipykernel_9708\3497306220.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


In [79]:
print(f"\nData range: {monthly_returns.index[0]} to {monthly_returns.index[-1]}")
print(f"Total number of monthly observations: {len(monthly_returns)}")


Data range: 2020-01-31 00:00:00 to 2025-08-31 00:00:00
Total number of monthly observations: 68


In [70]:
monthly_returns.cov()

Ticker,CSSPX.MI,EWJ,IEUR,VFEA.MI,VGEK.DE,WSML.L
Ticker,,,,,,
CSSPX.MI,0.002113,0.000974,0.001321,0.001192,0.001619,0.001951
EWJ,0.000974,0.001876,0.001995,0.001025,0.001476,0.001955
IEUR,0.001321,0.001995,0.003196,0.001448,0.002068,0.002763
VFEA.MI,0.001192,0.001025,0.001448,0.002087,0.001723,0.001792
VGEK.DE,0.001619,0.001476,0.002068,0.001723,0.002350,0.002480
WSML.L,0.001951,0.001955,0.002763,0.001792,0.002480,0.003428


In [71]:
asset_vol = monthly_returns.std() * np.sqrt(12)
asset_vol = asset_vol.rename('Ann.Vol')
asset_returns = (1 + monthly_returns.mean())**12 - 1  # Compound annualization
asset_sharpe = asset_returns / asset_vol

In [72]:
portfolio = rp.Portfolio(returns=monthly_returns)

portfolio.assets_stats(method_mu='hist', method_cov='hist')

bounds = (0.00, 0.60)
portfolio.assets_bounds = bounds

In [73]:
gmv = portfolio.optimization(model='Classic', rm='MV', obj='MinRisk', rf=0)
if gmv is not None:
    gmv.name = "Global Minimum Variance Portfolio"
    gmv_vol = (monthly_returns @ gmv.weights).std() * np.sqrt(12)
    gmv_sharpe = (monthly_returns @ gmv.weights).mean() / (monthly_returns @ gmv.weights).std() * np.sqrt(12)
    gmv_ret = (1 + (monthly_returns @ gmv.weights).mean())**12 - 1  # Compound annualization
else:
    print("GMV optimization failed")
    gmv_vol = gmv_sharpe = gmv_ret = np.nan

gms = portfolio.optimization(model='Classic', rm='MV', obj='Sharpe', rf=0)
if gms is not None:
    gms.name = "Global Maximum Sharpe Portfolio"
    gms_vol = (monthly_returns @ gms.weights).std() * np.sqrt(12)
    gms_sharpe = (monthly_returns @ gms.weights).mean() / (monthly_returns @ gms.weights).std() * np.sqrt(12)
    gms_ret = (1 + (monthly_returns @ gms.weights).mean())**12 - 1  # Compound annualization
else:
    print("GMS optimization failed")
    gms_vol = gms_sharpe = gms_ret = np.nan

gmr = portfolio.optimization(model='Classic', rm='MV', obj='MaxRet', rf=0)
if gmr is not None:
    gmr.name = "Global Maximum Return Portfolio"
    gmr_vol = (monthly_returns @ gmr.weights).std() * np.sqrt(12)
    gmr_sharpe = (monthly_returns @ gmr.weights).mean() / (monthly_returns @ gmr.weights).std() * np.sqrt(12)
    gmr_ret = (1 + (monthly_returns @ gmr.weights).mean())**12 - 1  # Compound annualization
else:
    print("GMR optimization failed")
    gmr_vol = gmr_sharpe = gmr_ret = np.nan

# Equal Weight Portfolio (1/N strategy)
ew_weights = pd.DataFrame(np.ones((len(tickers), 1)) / len(tickers), 
                         index=monthly_returns.columns, columns=['weights'])
ew_vol = (monthly_returns @ ew_weights.values).std() * np.sqrt(12)
ew_sharpe = (monthly_returns @ ew_weights.values).mean() / (monthly_returns @ ew_weights.values).std() * np.sqrt(12)
ew_ret = (1 + (monthly_returns @ ew_weights.values).mean())**12 - 1  # Compound annualization

In [74]:
# Asset metrics DataFrame
asset_results = pd.DataFrame({
    'Annualized_volatility': asset_vol,
    'Annualized_return': asset_returns,
    'Sharpe_ratio': asset_sharpe
})

# Portfolio metrics DataFrame
portfolio_results = pd.DataFrame({
    'Annualized_volatility_%': pd.Series({
        'Portfolio(GMV)': round(gmv_vol * 100, 2) if not np.isnan(gmv_vol) else None,
        'Portfolio(GMS)': round(gms_vol * 100, 2) if not np.isnan(gms_vol) else None,
        'Portfolio(GMR)': round(gmr_vol * 100, 2) if not np.isnan(gmr_vol) else None,
        'Portfolio(EW)': round(ew_vol[0] * 100, 2)
    }),
    'Annualized_return_%': pd.Series({
        'Portfolio(GMV)': round(gmv_ret * 100, 2) if not np.isnan(gmv_ret) else None,
        'Portfolio(GMS)': round(gms_ret * 100, 2) if not np.isnan(gms_ret) else None,
        'Portfolio(GMR)': round(gmr_ret * 100, 2) if not np.isnan(gmr_ret) else None,
        'Portfolio(EW)': round(ew_ret[0] * 100, 2)
    }),
    'Sharpe_ratio': pd.Series({
        'Portfolio(GMV)': round(gmv_sharpe, 3) if not np.isnan(gmv_sharpe) else None,
        'Portfolio(GMS)': round(gms_sharpe, 3) if not np.isnan(gms_sharpe) else None,
        'Portfolio(GMR)': round(gmr_sharpe, 3) if not np.isnan(gmr_sharpe) else None,
        'Portfolio(EW)': round(ew_sharpe[0], 3)
    })
})

# Add weights as percentages - accessing Series directly by ticker name
for ticker in monthly_returns.columns:
    portfolio_results[f'{ticker}_weight_%'] = pd.Series({
        'Portfolio(GMV)': round(gmv.weights[ticker] * 100, 2) if gmv is not None else None,
        'Portfolio(GMS)': round(gms.weights[ticker] * 100, 2) if gms is not None else None,
        'Portfolio(GMR)': round(gmr.weights[ticker] * 100, 2) if gmr is not None else None,
        'Portfolio(EW)': round(ew_weights.loc[ticker, 'weights'] * 100, 2)
    })

In [75]:
asset_results

,Annualized_volatility,Annualized_return,Sharpe_ratio
Ticker,,,
CSSPX.MI,0.159231,0.142511,0.894991
EWJ,0.150036,0.077933,0.519426
IEUR,0.195829,0.103379,0.527904
VFEA.MI,0.158245,0.049057,0.310004
VGEK.DE,0.167937,0.057360,0.341560
WSML.L,0.202827,0.095428,0.470492


In [76]:
portfolio_results

,Annualized_volatility_%,Annualized_return_%,Sharpe_ratio,CSSPX.MI_weight_%,EWJ_weight_%,IEUR_weight_%,VFEA.MI_weight_%,VGEK.DE_weight_%,WSML.L_weight_%
Portfolio(GMV),12.83,8.84,0.662,29.54,42.49,0.00,27.98,0.00,0.00
Portfolio(GMS),14.91,13.40,0.848,86.02,11.34,2.64,0.00,0.00,0.00
Portfolio(GMR),15.92,14.25,0.841,100.00,0.00,0.00,0.00,0.00,0.00
Portfolio(EW),14.90,8.72,0.563,16.67,16.67,16.67,16.67,16.67,16.67
